In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/ML_Assign_2/test.zip' -d '/content/test'
!unzip '/content/drive/MyDrive/ML_Assign_2/train.zip' -d '/content/train'
!unzip '/content/drive/MyDrive/ML_Assign_2/validation.zip' -d '/content/validation'

Streaming output truncated to the last 5000 lines.
  inflating: /content/train/train/NORMAL/NORMAL-8472707-54.jpeg  
  inflating: /content/train/train/NORMAL/NORMAL-8472707-55.jpeg  
  inflating: /content/train/train/NORMAL/NORMAL-8472707-7.jpeg  
  inflating: /content/train/train/NORMAL/NORMAL-8472707-9.jpeg  
  inflating: /content/train/train/NORMAL/NORMAL-8474778-13.jpeg  
  inflating: /content/train/train/NORMAL/NORMAL-8474778-16.jpeg  
  inflating: /content/train/train/NORMAL/NORMAL-8474778-17.jpeg  
  inflating: /content/train/train/NORMAL/NORMAL-8480965-13.jpeg  
  inflating: /content/train/train/NORMAL/NORMAL-8480965-9.jpeg  
  inflating: /content/train/train/NORMAL/NORMAL-8481138-5.jpeg  
  inflating: /content/train/train/NORMAL/NORMAL-8483291-9.jpeg  
  inflating: /content/train/train/NORMAL/NORMAL-8485328-4.jpeg  
  inflating: /content/train/train/NORMAL/NORMAL-8487067-20.jpeg  
  inflating: /content/train/train/NORMAL/NORMAL-8487067-25.jpeg  
  inflating: /content/train/tra

In [ ]:
!pip install keras-attention
!pip install custom_attention_module


ERROR: Could not find a version that satisfies the requirement custom_attention_module (from versions: none)
ERROR: No matching distribution found for custom_attention_module


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras.layers.attention.multi_head_attention import activation
IMAGE_SHAPE=(224,224)
BATCH_SIZE=32
EPOCHS=5

In [ ]:
train_dir="/content/train/train"
test_dir="/content/test"
val_dir="/content/validation/validation"

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255)
val_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
train_data = train_datagen.flow_from_directory(
    directory=train_dir,
    batch_size=32,
    target_size=(244, 244),
    class_mode="categorical",
    seed=42
)

test_data = test_datagen.flow_from_directory(
    directory=test_dir,
    batch_size=32,
    target_size=(244, 244),
    class_mode="categorical",
    seed=42
)

val_data = val_datagen.flow_from_directory(
    directory=val_dir,
    batch_size=32,
    target_size=(244, 244),
    class_mode="categorical",
    # seed=42
    shuffle=False
)


Found 20000 images belonging to 4 classes.
Found 10000 images belonging to 1 classes.
Found 4000 images belonging to 4 classes.


In [ ]:
def create_model(model_url,num_classes=10):
  feature_extractor_layer=hub.KerasLayer(model_url,trainable=False,
                                         name="feature_extraction_layer",
                                         input_shape=IMAGE_SHAPE+(3,))
  model=tf.keras.Sequential([
      feature_extractor_layer,
      layers.Dense(num_classes,
                   activation="softmax",
                   name="out_layer"),
  ])

  return model

In [ ]:
resnet_url="https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

efficientnet_url="https://tfhub.dev/google/efficientnet/b0/feature-vector/1"

inception_url="https://tfhub.dev/google/logit_reconstruction/inceptionv3/1"

In [ ]:
train_data.num_classes

4

**RESNET_V2_50 model**

In [ ]:
resnet_model=create_model(resnet_url,
                          num_classes=4)

In [ ]:
from tensorflow.keras.metrics import Precision, F1Score , Recall
resnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy",Precision(),Recall()])

In [ ]:
resnet_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 2048)              23564800  
                                                                 
 dense_1 (Dense)             (None, 4)                 8196      
                                                                 
Total params: 23572996 (89.92 MB)
Trainable params: 8196 (32.02 KB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________


In [ ]:
resnet_history=resnet_model.fit(train_data,
                                epochs=2,
                                validation_data=val_data,
                                batch_size=32
                                )

Epoch 1/2
625/625 [==============================] - 129s 204ms/step - loss: 0.5789 - accuracy: 0.7747 - precision_2: 0.8200 - recall_1: 0.7205 - val_loss: 0.5233 - val_accuracy: 0.8018 - val_precision_2: 0.8317 - val_recall_1: 0.7535
Epoch 2/2
625/625 [==============================] - 115s 184ms/step - loss: 0.4875 - accuracy: 0.8161 - precision_2: 0.8501 - recall_1: 0.7766 - val_loss: 0.4964 - val_accuracy: 0.8080 - val_precision_2: 0.8342 - val_recall_1: 0.7738


In [ ]:
res_preds=resnet_model.predict(val_data,steps=len(val_data))
res_preds

125/125 [==============================] - 19s 150ms/step


array([[8.9132553e-01, 9.1080731e-03, 9.9418268e-02, 1.4802991e-04],
       [9.2985833e-01, 4.0007126e-02, 3.0070186e-02, 6.4285006e-05],
       [9.7950900e-01, 5.8845184e-03, 1.4458461e-02, 1.4794135e-04],
       ...,
       [3.8339989e-05, 3.9266266e-02, 3.5549510e-02, 9.2514592e-01],
       [1.3667386e-04, 2.3292804e-02, 9.2982262e-02, 8.8358819e-01],
       [2.7084523e-03, 3.3906106e-02, 2.8206387e-01, 6.8132156e-01]],
      dtype=float32)

In [ ]:
len(res_preds)

4000

In [ ]:
import numpy as np
pred_labels=np.argmax(res_preds,axis=1)
pred_labels

array([0, 0, 0, ..., 3, 3, 3])

In [ ]:
val_data.classes

array([0, 0, 0, ..., 3, 3, 3], dtype=int32)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(val_data.classes,pred_labels)
accuracy

0.808

**EFFICIENTNET_Model**

In [ ]:
efficientnet_model=create_model(efficientnet_url,
                          num_classes=4)

efficientnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy",Precision(),Recall()])

efficientnet_history=efficientnet_model.fit(train_data,
                                epochs=2,
                                validation_data=val_data,
                                batch_size=32
                                )

Epoch 1/2
625/625 [==============================] - 100s 153ms/step - loss: 0.6638 - accuracy: 0.7601 - precision_3: 0.8527 - recall_2: 0.6222 - val_loss: 0.5183 - val_accuracy: 0.8165 - val_precision_3: 0.8661 - val_recall_2: 0.7520
Epoch 2/2
625/625 [==============================] - 94s 150ms/step - loss: 0.4744 - accuracy: 0.8342 - precision_3: 0.8706 - recall_2: 0.7828 - val_loss: 0.4572 - val_accuracy: 0.8347 - val_precision_3: 0.8697 - val_recall_2: 0.7910


In [ ]:
eff_preds=efficientnet_model.predict(val_data,steps=len(val_data))
eff_preds

125/125 [==============================] - 16s 125ms/step


array([[4.4053665e-01, 4.3602681e-01, 1.2290252e-01, 5.3406548e-04],
       [4.4029570e-01, 5.5174708e-01, 7.9003517e-03, 5.6899971e-05],
       [9.7129041e-01, 2.5594980e-02, 3.1112696e-03, 3.2544065e-06],
       ...,
       [2.0901921e-04, 2.2346655e-02, 7.7266976e-02, 9.0017736e-01],
       [2.8007198e-03, 5.2709258e-01, 1.5277129e-01, 3.1733546e-01],
       [3.5791930e-03, 4.5207828e-02, 2.4134050e-01, 7.0987248e-01]],
      dtype=float32)

In [ ]:
eff_pred_labels=np.argmax(eff_preds,axis=1)
eff_pred_labels

array([0, 1, 0, ..., 3, 1, 3])

In [ ]:
from sklearn.metrics import accuracy_score
eff_accuracy=accuracy_score(val_data.classes,eff_pred_labels)
eff_accuracy

0.83475

**As EfficientNet is showing more accuracy, Hence, We will use EfficientNet for the test data output prediction**

In [ ]:
Final_preds=efficientnet_model.predict(test_data,steps=len(val_data))
Final_preds

125/125 [==============================] - 22s 178ms/step


array([[3.2870469e-04, 3.7690919e-02, 1.7117379e-02, 9.4486302e-01],
       [2.5308554e-03, 2.5801286e-01, 1.4988860e-02, 7.2446740e-01],
       [8.4549779e-01, 2.6311049e-02, 1.2803666e-01, 1.5445919e-04],
       ...,
       [6.3014650e-01, 3.6816803e-01, 1.5354053e-03, 1.5007016e-04],
       [1.6052051e-03, 7.1092382e-02, 1.2707254e-01, 8.0022997e-01],
       [7.6025509e-04, 6.1642550e-02, 1.1251019e-01, 8.2508707e-01]],
      dtype=float32)

In [ ]:
Final_labels=np.argmax(Final_preds,axis=1)
Final_labels #Here output will be an array in which 0 is showing CNV, 1 is showing DME, 2 is showing DRUSEN and 3 is showing NORMAL

array([3, 3, 0, ..., 0, 3, 3])

**INCEPTION_V2_MODEL**
I tried inception model but don't know why the RAM is crashing every time.

In [ ]:

inception_model=create_model(inception_url,
                          num_classes=4)

inception_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy",Precision(),Recall()])

inception_history=inception_model.fit(train_data,
                                epochs=2,
                                validation_data=val_data,
                                )